
## Overview

Esse notebook cria uma tabela permanente no DBFS com os dados de Cartão Corporativo do Governo Federal. Esta tabela é considerada a tabela 'fato', pois contém os valores das transações. A tabela também foi normalizada para compor o modelo 'estrela'. 

In [0]:
from datetime import datetime
from pyspark.sql.functions import col, date_format

In [0]:
# File location and type
file_location = "/FileStore/tables/202401_CPGF-1.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"
encode = 'latin1'

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("encoding", encode)  \
  .load(file_location)

# display(df)

In [0]:
cols_to_drop = ["NOME ÓRGÃO SUPERIOR", "NOME ÓRGÃO", "NOME UNIDADE GESTORA"]
df = df.drop(*cols_to_drop)


In [0]:
df = df.withColumn("DATA TRANSAÇÃO", date_format(col("DATA TRANSAÇÃO"), "dd/MM/yyyy"))

In [0]:
df = df.to_pandas_on_spark()

/databricks/spark/python/pyspark/sql/dataframe.py:5178: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


In [0]:
# df["DATA TRANSAÇÃO"] = df["DATA TRANSAÇÃO"].apply(lambda x: datetime.strptime(str(x), "%d/%m/%Y") if x not null else str(x))

In [0]:
df["VALOR TRANSAÇÃO"] = df["VALOR TRANSAÇÃO"].apply(lambda x: float(str(x).replace(",", ".")))

In [0]:
df = df[(df["CNPJ OU CPF FAVORECIDO"] != -1) & (df["CNPJ OU CPF FAVORECIDO"] != -2)]

In [0]:
column_names = ["cod_org_sup", "cod_org", "unid_gestora", 
                "ano_extrato", "mes_extrato", "cpf_portador", 
                "nome_portador", "cnpj_cpf_fav", "nome_fav",
                "transaction", "dt_trans", "vlr_trans"]

df.columns = column_names

In [0]:
df[["cod_org_sup", "cod_org", "unid_gestora"]] = df[["cod_org_sup", "cod_org", "unid_gestora"]].astype(int)

In [0]:
df = df.to_spark()

In [0]:
%sql
DROP TABLE IF EXISTS fato_cpgf

In [0]:
# cpgf -> cartão de pagamentos do governo federal
permanent_table_name = "fato_cpgf"
# df.write.format('parquet').saveAsTable(permanent_table_name)
df.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
%sql
SELECT
    fc.cod_org, ol.ORG_LOTACAO,     
    ROUND(SUM(fc.vlr_trans), 2) AS total_gasto
FROM
    fato_cpgf fc
INNER JOIN 
    org_lotacao ol 
ON
    fc.cod_org = ol.COD_ORG_LOTACAO
GROUP BY
    fc.cod_org, ol.ORG_LOTACAO
ORDER BY
    total_gasto DESC
LIMIT 10

cod_org,ORG_LOTACAO,total_gasto
20101,Presidência da República,2221202.95
25000,Ministério da Saúde,100584.24
30802,Departamento de Polícia Rodoviária Federal,67798.53
36201,Fundação Oswaldo Cruz,47141.75
26429,Instituto Federal de Goiás,46737.75
26443,Empresa Brasileira de Serviços Hospitalares,42848.09
32396,Agência Nacional de Mineração,38104.58
44207,Instituto Chico Mendes de Conservação da Biodiversidade,34163.0
26249,Universidade Federal Rural do Rio de Janeiro,29963.61
26263,Universidade Federal de Lavras - MG,28829.16


In [0]:
%sql
SELECT 
  fc.nome_portador, ug.UORG_LOTACAO, 
  round(sum(fc.vlr_trans), 2) AS total_gasto
FROM 
  fato_cpgf fc
INNER JOIN
  cadastro_servidores cs
ON
  fc.cpf_portador = cs.CPF
INNER JOIN
  qualificacao_servidores qs
ON
  qs.Id_SERVIDOR_PORTAL = cs.id_SERVIDOR_PORTAL
INNER JOIN
  unid_org ug
ON
  qs.COD_UORG_LOTACAO = ug.COD_UORG_LOTACAO
GROUP BY
  fc.nome_portador, ug.UORG_LOTACAO
ORDER BY
  total_gasto DESC
LIMIT 10

nome_portador,UORG_LOTACAO,total_gasto
ANTENOR FONSECA DE OLIVEIRA FILHO,CENTRO DE REGISTRO E CONTROLE ACADEMICO,40233.66
NISVAL FERREIRA GUIMARAES,DIRECAO GERAL CAMPUS INHUMAS,38000.58
RICARDO YANEZ NOGUEIRA,INSTITUTO NAC DE CONTROL QUALID EM SAUDE,27917.96
DAVID CEZARIO DE SOUZA FILHO,INSTITUTO DE TECNOLOGIA EM IMUNOBIOLOGIC,27250.36
FERNANDO FRANCISCO DE SOUZA,DIRECAO GERAL CAMPUS INHUMAS,24808.42
MARCIA MARIA HENGEMUHLE,HOSPITAL UNIVERSITARIO DA UFSM,24260.94
LEONARDO LIMA BERGAMINI,GERENCIA DE MEIO AMBIENTE E GEOGRAFIA,21431.98
PAULO CESAR DO NASCIMENTO CORREA,COORDENACAO GERAL INFRAESTRUTURA CAMPI,21417.6
ANTENOR FONSECA DE OLIVEIRA FILHO,DIRETORIA DE PLANEJAMENTO,20116.83
ANTENOR FONSECA DE OLIVEIRA FILHO,DIVISAO OPERACIONAL,20116.83


In [0]:
%sql
SELECT
    fc.cod_org, ol.ORG_LOTACAO, 
    ROUND(SUM(fc.vlr_trans), 2) AS total_gasto
FROM
    fato_cpgf fc
INNER JOIN 
    org_lotacao ol 
ON
    fc.cod_org = ol.COD_ORG_LOTACAO
GROUP BY
    fc.cod_org, ol.ORG_LOTACAO
ORDER BY
    total_gasto DESC

cod_org,ORG_LOTACAO,total_gasto
20101,Presidência da República,2221202.95
25000,Ministério da Saúde,100584.24
30802,Departamento de Polícia Rodoviária Federal,67798.53
36201,Fundação Oswaldo Cruz,47141.75
26429,Instituto Federal de Goiás,46737.75
26443,Empresa Brasileira de Serviços Hospitalares,42848.09
32396,Agência Nacional de Mineração,38104.58
44207,Instituto Chico Mendes de Conservação da Biodiversidade,34163.0
26249,Universidade Federal Rural do Rio de Janeiro,29963.61
26263,Universidade Federal de Lavras - MG,28829.16


In [0]:
%sql
WITH avgspending AS (
    SELECT
        ROUND(AVG(total_gasto), 2) AS avg_gasto
    FROM (
        SELECT
            fc.cod_org,
            ol.ORG_LOTACAO,
            SUM(fc.vlr_trans) AS total_gasto
        FROM
            fato_cpgf fc
        INNER JOIN 
            org_lotacao ol 
        ON
            fc.cod_org = ol.COD_ORG_LOTACAO
        GROUP BY
            fc.cod_org,
            ol.ORG_LOTACAO
  )
)

SELECT
    fc.cod_org,
    ol.ORG_LOTACAO,
    ROUND(SUM(fc.vlr_trans), 2) AS total_gasto
FROM
    fato_cpgf fc
INNER JOIN 
    org_lotacao ol 
ON
    fc.cod_org = ol.COD_ORG_LOTACAO
GROUP BY
    fc.cod_org,
    ol.ORG_LOTACAO
HAVING
    SUM(fc.vlr_trans) > (SELECT avg_gasto FROM avgspending)
ORDER BY
    total_gasto DESC

cod_org,ORG_LOTACAO,total_gasto
20101,Presidência da República,2221202.95
25000,Ministério da Saúde,100584.24
30802,Departamento de Polícia Rodoviária Federal,67798.53
36201,Fundação Oswaldo Cruz,47141.75
26429,Instituto Federal de Goiás,46737.75
26443,Empresa Brasileira de Serviços Hospitalares,42848.09
